In [94]:
using JuMP, Gurobi, JSON, DataFrames
# JuMP v1.13.0
# Julia v1.8.5
# Gurobi v1.0.1
# JSON v0.21.4
# DataFrame v1.5.0


# This system looks like:
# Silo1 -> Machine1 -> Silo2 -> Machine2 -> Silo3
#       -5         +10      -10         +20
#                 ^               ^
#                 \              /
#                    cleaner1    

# in this example cleaning is not relevant but all the code is still included 
# there is one cleaner and it can only clean one machine at a time
# I've tried to simplify the model as much as possible so it should be easier to understand

In [95]:
# this code is all just reading in info from the jsons and putting them into a struct
# code isn't relevant to the bug

struct Data
  resources
  machines
  states
  periods
end

resource_data = JSON.parsefile("resources.json")
machine_data = JSON.parsefile("machines.json")
states_data = JSON.parsefile("states.json")

resources = [(name = resource_name,
          min_capacity = resource_data["min_capacity"],
          max_capacity = resource_data["max_capacity"],
          initial_volume = resource_data["initial_volume"],
          rates = resource_data["rates"],
          flows = resource_data["flows"],
          type = resource_data["type"])
          for (resource_name, resource_data) in resource_data]

machines = [(name = machine_name,
             initial_state = machine_data["initial_state"],
             initial_time = machine_data["initial_time"],
             run_time = machine_data["run_time"],
             cleaning_time = machine_data["cleaning_time"],
             min_off_time = machine_data["min_off_time"],
             max_off_time = machine_data["max_off_time"],
             rates = machine_data["rates"],
             flows = machine_data["flows"],
             cleaner_key = get(machine_data, "cleaner_key", "NA"))
             for (machine_name, machine_data) in machine_data]

states = [(current_state = current_state,
              next_state = state_data["next_state"],
              duration_type = state_data["duration_type"],
              duration_key = get(state_data, "duration_key", "NA"),
              min_duration_key = get(state_data, "min_duration_key", "NA"),
              max_duration_key = get(state_data, "max_duration_key", "NA"),
              include = state_data["include"]) 
              for (current_state, state_data) in states_data]

𝓓 = Data(resources, machines, states, 5)

Data(NamedTuple{(:name, :min_capacity, :max_capacity, :initial_volume, :rates, :flows, :type), Tuple{String, Int64, Int64, Int64, Vector{Any}, Vector{Any}, String}}[(name = "silo1", min_capacity = 0, max_capacity = 1000, initial_volume = 900, rates = [], flows = [], type = "storage"), (name = "silo3", min_capacity = 0, max_capacity = 600, initial_volume = 300, rates = [], flows = [], type = "storage"), (name = "silo2", min_capacity = 0, max_capacity = 500, initial_volume = 200, rates = [], flows = [], type = "storage"), (name = "cleaner1", min_capacity = 0, max_capacity = 1, initial_volume = 1, rates = [-1, -1], flows = ["machine1", "machine2"], type = "renewable")], NamedTuple{(:name, :initial_state, :initial_time, :run_time, :cleaning_time, :min_off_time, :max_off_time, :rates, :flows, :cleaner_key), Tuple{String, String, Int64, Int64, Int64, Int64, Int64, Vector{Any}, Vector{Any}, String}}[(name = "machine2", initial_state = "cleaning", initial_time = 1, run_time = 7, cleaning_time 

In [96]:
# I am "generating" a schedule for each machine (for our example they are hard coded for reproducability)

itteration = 1
schedules_dict = Dict()
schedules_dict["machine1", itteration] = ["on", "on", "on", "on", "on"]
schedules_dict["machine2", itteration] = ["on", "on", "on", "on", "on"]
schedules_dict

Dict{Any, Any} with 2 entries:
  ("machine1", 1) => ["on", "on", "on", "on", "on"]
  ("machine2", 1) => ["on", "on", "on", "on", "on"]

In [97]:
𝓜 = Model(Gurobi.Optimizer)

# binary variable to select which schedule to use
x = Dict{String, Vector{VariableRef}}(
  d.name => @variable(𝓜, [1:1], binary = true, base_name = "$(d.name)")
  for d in 𝓓.machines
)

# Variable to track resource volume
@variable(𝓜, resource_volume[p in 0:𝓓.periods, [r.name for r in 𝓓.resources]])

# Constraint to ensure solver picks a schedule value, because selecting nothing is always feasible
GUB_dict = Dict(m.name => @constraint(𝓜, sum(x[m.name][1]) == 1) for m in 𝓓.machines)

# initial resource volume constraint
for r in 𝓓.resources
  @constraint(𝓜, resource_volume[0, r.name] == r.initial_volume)
end

# resource volume constraint for each time period below
#
# I am building an expresion that represents the rate of change each a resource experiences at a given time
# according to the machines that interact with it
resource_volume_con_dict = Dict()
resource_volume_expr_dict = Dict()
for p in 1:𝓓.periods, r in 𝓓.resources
  rate_expression = AffExpr()

  for m in 𝓓.machines
    machine_activity = schedules_dict[m.name, 1][p]
            
    if r.name in m.flows && machine_activity == "on"
      index = findfirst(x -> x == r.name, m.flows)
      add_to_expression!(rate_expression, m.rates[index], x[m.name][1])

    elseif  r.name == m.cleaner_key && machine_activity == "cleaning"
      index = findfirst(x -> x == m.name, r.flows)
      add_to_expression!(rate_expression, r.rates[index], x[m.name][1])
    end
  end

  # after building the rate expression I build the constraint that represent how the resource volume
  # changes over time depending on the type of resource it is
  # at the moment we are storing the rate expression in a dict for a workaround to the bug
  if r.type == "storage"
    resource_volume_con_dict[p, r.name] = @constraint(𝓜, resource_volume[p, r.name] - resource_volume[p-1, r.name] - rate_expression == 0)
    resource_volume_expr_dict[p, r.name] = rate_expression
  end

  if r.type == "renewable"
    resource_volume_con_dict[p, r.name] = @constraint(𝓜, resource_volume[p, r.name] - r.max_capacity - rate_expression == 0)
    resource_volume_expr_dict[p, r.name] = rate_expression
  end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-07


In [98]:
# These are normalised constraints it is easier to think of it like:
# resource_volume[1,silo1] == resource_volume[0,silo1] -5*machine1[1]
# current_time_volume = previous_time_volume + rate_of_change_at_current_time*relevant_schedules_binary_variable
print(𝓜)

Feasibility
Subject to
 machine2[1] == 1
 machine1[1] == 1
 resource_volume[0,silo1] == 900
 resource_volume[0,silo3] == 300
 resource_volume[0,silo2] == 200
 resource_volume[0,cleaner1] == 1
 5 machine1[1] - resource_volume[0,silo1] + resource_volume[1,silo1] == 0
 -20 machine2[1] - resource_volume[0,silo3] + resource_volume[1,silo3] == 0
 10 machine2[1] - 10 machine1[1] - resource_volume[0,silo2] + resource_volume[1,silo2] == 0
 resource_volume[1,cleaner1] == 1
 5 machine1[1] - resource_volume[1,silo1] + resource_volume[2,silo1] == 0
 -20 machine2[1] - resource_volume[1,silo3] + resource_volume[2,silo3] == 0
 10 machine2[1] - 10 machine1[1] - resource_volume[1,silo2] + resource_volume[2,silo2] == 0
 resource_volume[2,cleaner1] == 1
 5 machine1[1] - resource_volume[2,silo1] + resource_volume[3,silo1] == 0
 -20 machine2[1] - resource_volume[2,silo3] + resource_volume[3,silo3] == 0
 10 machine2[1] - 10 machine1[1] - resource_volume[2,silo2] + resource_volume[3,silo2] == 0
 resource_volu

In [99]:
optimize!(𝓜)
println(termination_status(𝓜))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 26 columns and 61 nonzeros
Model fingerprint: 0x36ccc6bf
Variable types: 24 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 26 rows and 26 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

User-callback calls 121, time in user-callback 0.00 sec
OPTIMAL


In [100]:
# Code to figure out which schedule (binary varible) was selected for each machine
# none of this is relevant to the bug but nice to visualise what is going on
selected_schedule = Dict{String, Int64}()
for m in 𝓓.machines
    for i in 1:length(x[m.name])
      if value(x[m.name][i]) == 1
          selected_schedule[m.name] = i
      end
    end
end
     
df = DataFrame(period=[], silo1=[], machine1=[],  silo2=[], machine2=[], silo3=[], cleaner1 = [])
for p in 1:𝓓.periods
    push!(df, [p, value(resource_volume[p, "silo1"]), schedules_dict["machine1", selected_schedule["machine1"]][p], value(resource_volume[p, "silo2"]), schedules_dict["machine2", selected_schedule["machine2"]][p], value(resource_volume[p, "silo3"]), value(resource_volume[p, "cleaner1"])])
end
display(df)

Row,period,silo1,machine1,silo2,machine2,silo3,cleaner1
,Any,Any,Any,Any,Any,Any,Any
1,1,895.0,on,200.0,on,320.0,1.0
2,2,890.0,on,200.0,on,340.0,1.0
3,3,885.0,on,200.0,on,360.0,1.0
4,4,880.0,on,200.0,on,380.0,1.0
5,5,875.0,on,200.0,on,400.0,1.0


In [101]:
# the above output is what we expect to see
# I am adding the exact same schedule just to illustrate the bug

itteration = 2
schedules_dict["machine1", itteration] = ["on", "on", "on", "on", "on"]
schedules_dict["machine2", itteration] = ["on", "on", "on", "on", "on"]
schedules_dict

Dict{Any, Any} with 4 entries:
  ("machine2", 2) => ["on", "on", "on", "on", "on"]
  ("machine1", 2) => ["on", "on", "on", "on", "on"]
  ("machine1", 1) => ["on", "on", "on", "on", "on"]
  ("machine2", 1) => ["on", "on", "on", "on", "on"]

In [102]:
# this is where the bug is encountered. What is happening here is we create a new binary variable for each machine
# that represents a new schedule (in our example the exact same schedule) 

# if I use set_normalized_coefficient to update the model then the bug is encountered
# The bug is the coefficents associated with the new binary variables are applied to the wrong constraint on solve
# if you look at the model before solve everything looks like it should 
# but after solve it seems like the coefficents (rates) are applied to the wrong constraint (resource)

# our current workaround is to just delete the constraint and re-add it 
# we are using dictionaries to find the correct constraint reference and expression that represent the constraint


itteration = 2
force_this_schedule = true # special gub constraint to force our new schedule (binary variable) to be selected
bug = true # boolean to use the buggy method or the workaround

# create a new binary variable that represents a schedule for each machine
for m in 𝓓.machines
    push!(x[m.name], @variable(𝓜, [[itteration]], binary = true, base_name = "$(m.name)")[itteration])
end

# Update GUB constraint
for m in 𝓓.machines
  # delete old GUB constraint
  delete(𝓜, GUB_dict[m.name])
  GUB_dict[m.name] = @constraint(𝓜, sum(x[m.name][1:itteration]) == 1)

  # our special GUB const to force this schedule to be selected
  force_this_schedule ? @constraint(𝓜, sum(x[m.name][itteration]) == 1) : nothing
end

# Update resource volume constraint
for p in 1:𝓓.periods, r in 𝓓.resources
  rate_expression = resource_volume_expr_dict[p, r.name]

  for m in 𝓓.machines
    machine_activity = schedules_dict[m.name, itteration][p]
          
    if r.name in m.flows && machine_activity == "on"
      index = findfirst(x -> x == r.name, m.flows)
      if !bug
        add_to_expression!(rate_expression, m.rates[index], x[m.name][itteration])
      else
        # This is the where I think the source of the bug is
        set_normalized_coefficient(resource_volume_con_dict[p, r.name], x[m.name][itteration], -m.rates[index])
      end

    elseif  r.name == m.cleaner_key && machine_activity == "cleaning"
      index = findfirst(x -> x == m.name, r.flows)
      if !bug
        add_to_expression!(rate_expression, r.rates[index], x[m.name][itteration])
      else
        # This is the where I think the source of the bug is
        set_normalized_coefficient(resource_volume_con_dict[p, r.name], x[m.name][itteration], -r.rates[index])
      end
    end
  end

  # This code is not relevant to the bug but needed for our workaround
  if r.type == "storage" && !bug
    delete(𝓜, resource_volume_con_dict[p, r.name])
    resource_volume_con_dict[p, r.name] = @constraint(𝓜, resource_volume[p, r.name] - resource_volume[p-1, r.name] - rate_expression == 0)
    resource_volume_expr_dict[p, r.name] = rate_expression

  elseif r.type == "renewable" && !bug
    delete(𝓜, resource_volume_con_dict[p, r.name])
    resource_volume_con_dict[p, r.name] = @constraint(𝓜, resource_volume[p, r.name] - r.max_capacity - rate_expression == 0)
    resource_volume_expr_dict[p, r.name] = rate_expression
  end
end


In [103]:
# investigating the .lp files for both methods (bug or workaround) the model seems to be almost identical
# the only difference I can notice is the ordering 
bug ? write_to_file(𝓜, "bug.lp") : write_to_file(𝓜, "normal.lp")

In [104]:
optimize!(𝓜)
println(termination_status(𝓜))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 28 columns and 85 nonzeros
Model fingerprint: 0x12fa0fc4
Variable types: 24 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+02]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 28 rows and 28 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

User-callback calls 153, t

In [105]:
# These constraint are what we expect to see (no matter whether we choose buggy method or not)
print(𝓜)

Feasibility
Subject to
 resource_volume[0,silo1] == 900
 resource_volume[0,silo3] == 300
 resource_volume[0,silo2] == 200
 resource_volume[0,cleaner1] == 1
 5 machine1[1] - resource_volume[0,silo1] + resource_volume[1,silo1] + 5 machine1[2] == 0
 -20 machine2[1] - resource_volume[0,silo3] + resource_volume[1,silo3] - 20 machine2[2] == 0
 10 machine2[1] - 10 machine1[1] - resource_volume[0,silo2] + resource_volume[1,silo2] + 10 machine2[2] - 10 machine1[2] == 0
 resource_volume[1,cleaner1] == 1
 5 machine1[1] - resource_volume[1,silo1] + resource_volume[2,silo1] + 5 machine1[2] == 0
 -20 machine2[1] - resource_volume[1,silo3] + resource_volume[2,silo3] - 20 machine2[2] == 0
 10 machine2[1] - 10 machine1[1] - resource_volume[1,silo2] + resource_volume[2,silo2] + 10 machine2[2] - 10 machine1[2] == 0
 resource_volume[2,cleaner1] == 1
 5 machine1[1] - resource_volume[2,silo1] + resource_volume[3,silo1] + 5 machine1[2] == 0
 -20 machine2[1] - resource_volume[2,silo3] + resource_volume[3,silo

In [106]:
# Here is the symptom of bug
# Looking at silo1 it is increasing by 20 each time period but if we read the constraint that should not be possible
# it's weird in that the +20 comes from the rate that should be being applied to silo3 according to the constraints

# Same goes for the cleaner, the -5 that should be being applied to silo1 is being applied to cleaner1
# Also looking at the cleaner constraints e.g.
# resource_volume[1,cleaner1] == 1
# resource_volume[2,cleaner1] == 1
# resource_volume[3,cleaner1] == 1
# resource_volume[4,cleaner1] == 1
# these constraint are very obviously being violated as seen in the below dataframe (as indicated by the -4)

# Code to figure out which schedule was selected 
for m in 𝓓.machines
    for i in 1:length(x[m.name])
      if value(x[m.name][i]) == 1
          selected_schedule[m.name] = i
      end
    end
end
     
df = DataFrame(period=[], silo1=[], machine1=[],  silo2=[], machine2=[], silo3=[], cleaner1 = [])
for p in 1:𝓓.periods
    push!(df, [p, value(resource_volume[p, "silo1"]), schedules_dict["machine1", selected_schedule["machine1"]][p], value(resource_volume[p, "silo2"]), schedules_dict["machine2", selected_schedule["machine2"]][p], value(resource_volume[p, "silo3"]), value(resource_volume[p, "cleaner1"])])
end
display(df)

# Also bear in mind that this should look like the previous itteration if everything is working correctly 

Row,period,silo1,machine1,silo2,machine2,silo3,cleaner1
,Any,Any,Any,Any,Any,Any,Any
1,1,920.0,on,200.0,on,300.0,-4.0
2,2,940.0,on,200.0,on,300.0,-4.0
3,3,960.0,on,200.0,on,300.0,-4.0
4,4,980.0,on,200.0,on,300.0,-4.0
5,5,1000.0,on,200.0,on,300.0,1.0
